In [1]:
#!git clone https://github.com/aub-mind/arabert

In [2]:
#!pip install PyArabic

In [55]:
import numpy as np
import pandas as pd
import nltk
from arabert.preprocess import ArabertPreprocessor
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report

In [2]:
df_train_pos = pd.read_csv('D:\\Sentiment Analysis\\Data\\train_Arabic_tweets_positive_20190413.tsv', sep = '\t', header=None)
df_train_neg = pd.read_csv('D:\\Sentiment Analysis\\Data\\train_Arabic_tweets_negative_20190413.tsv', sep = '\t', header=None)
train_df = pd.concat([df_train_pos, df_train_neg],ignore_index=True)
train_df.set_axis(['target', 'tweet'], axis=1, inplace=True)

In [3]:
train_df['target'].value_counts()

pos    22761
neg    22514
Name: target, dtype: int64

In [5]:
train_df.head()

,target,tweet
0,pos,نحن الذين يتحول كل ما نود أن نقوله إلى دعاء لل...
1,pos,وفي النهاية لن يبقىٰ معك آحدإلا من رأىٰ الجمال...
2,pos,من الخير نفسه 💛
3,pos,#زلزل_الملعب_نصرنا_بيلعب كن عالي الهمه ولا ترض...
4,pos,الشيء الوحيد الذي وصلوا فيه للعالمية هو : المس...


In [6]:
train_df.tweet.isna().value_counts()

False    45275
Name: tweet, dtype: int64

In [11]:
model_name = 'aubmindlab/bert-base-arabertv02'
arabert_prep = ArabertPreprocessor(model_name.split("/")[-1])
train_df['processed_text'] = train_df.tweet.apply(lambda x: arabert_prep.preprocess(x))

In [12]:
train_df = train_df.sample(frac=1)

In [13]:
train_df.head()

,target,tweet,processed_text
16155,pos,#نطالب_بفتح_مكبرات_الصوت 🔹 رفع نداء الحق في مك...,# نطالب _ بفتح _ مكبرات _ الصوت رفع نداء الحق ...
14027,pos,ما بس للتأكيد يعني 😂,ما بس للتأكيد يعني
40790,neg,الكل جالس يقرأ كميتسو 🤐,الكل جالس يقرأ كميتسو
16041,pos,اللهم آمين 🌼,اللهم آمين
13253,pos,ما شاء الله تبارك الله 👍,ما شاء الله تبارك الله


In [14]:
data = np.array(train_df['processed_text'])
data_l = np.array(train_df['target'])

In [15]:
data_l[2]

'neg'

In [16]:
for i in range(len(data_l)):
    if data_l[i] == 'pos': data_l[i] = 1
    else : data_l[i] = 0

In [17]:
data_l

array([1, 1, 0, ..., 0, 1, 1], dtype=object)

In [18]:
def data_pre(tweet):
    
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(tweet)
    stopwords_ara = set(stopwords.words('arabic'))
    clean_text = [token for token in tokens if token not in stopwords_ara]
    clean_text = " ".join(clean_text)
    return clean_text

In [19]:
traning = [data_pre(i) for i in data]
#testing = [data_pre(i) for i in X_test]

In [20]:
cv = CountVectorizer()

In [21]:
X_vec = cv.fit_transform(traning).toarray()

In [22]:
len(X_vec)

45275

In [24]:
t_data = X_vec[:36220]
t_label = data_l[:36220]
tt_data = X_vec[36220:]
tt_label = data_l[36220:]

In [30]:
len(tt_label)

9055

In [33]:
from sklearn.naive_bayes import MultinomialNB

In [34]:
mn = MultinomialNB()

In [35]:
mn.fit(t_data, list(t_label))

MultinomialNB()

In [36]:
pred = mn.predict(tt_data)

In [57]:
print(classification_report(list(tt_label), pred))

              precision    recall  f1-score   support

           0       0.78      0.79      0.78      4562
           1       0.78      0.77      0.77      4493

    accuracy                           0.78      9055
   macro avg       0.78      0.78      0.78      9055
weighted avg       0.78      0.78      0.78      9055



In [58]:
from sklearn.svm import LinearSVC
svc = LinearSVC()
svc.fit(t_data, list(t_label))

C:\Users\engse\anaconda3\envs\tf_2.7\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


LinearSVC()

In [59]:
pred1 = svc.predict(tt_data)

In [60]:
print(classification_report(list(tt_label), pred1))

              precision    recall  f1-score   support

           0       0.77      0.79      0.78      4562
           1       0.78      0.76      0.77      4493

    accuracy                           0.78      9055
   macro avg       0.78      0.78      0.78      9055
weighted avg       0.78      0.78      0.78      9055



In [62]:
from sklearn.ensemble import RandomForestClassifier
RFM = RandomForestClassifier(n_estimators = 3)
RFM.fit(t_data, list(t_label))

RandomForestClassifier(n_estimators=3)

In [63]:
pred2 = RFM.predict(tt_data)

In [64]:
print(classification_report(list(tt_label), pred2))

              precision    recall  f1-score   support

           0       0.75      0.80      0.77      4562
           1       0.78      0.73      0.75      4493

    accuracy                           0.76      9055
   macro avg       0.76      0.76      0.76      9055
weighted avg       0.76      0.76      0.76      9055

